In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import os
import io
import glob

In [2]:
!wget "https://drive.google.com/uc?export=download&id=1vzvJj-fXsnuBHVe0mZXu-0gNv-bTgfiZ" -O HandDB_Lower_Alphabet.npz

--2022-09-13 14:51:40--  https://drive.google.com/uc?export=download&id=1vzvJj-fXsnuBHVe0mZXu-0gNv-bTgfiZ
Resolving drive.google.com (drive.google.com)... 74.125.195.138, 74.125.195.113, 74.125.195.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.195.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3r7v30hlaealtc73grq0ijpcsd9me72a/1663080675000/12360162376384578551/*/1vzvJj-fXsnuBHVe0mZXu-0gNv-bTgfiZ?e=download&uuid=af0debad-0770-4364-9e2a-fa437d149e2a [following]
--2022-09-13 14:52:06--  https://doc-14-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3r7v30hlaealtc73grq0ijpcsd9me72a/1663080675000/12360162376384578551/*/1vzvJj-fXsnuBHVe0mZXu-0gNv-bTgfiZ?e=download&uuid=af0debad-0770-4364-9e2a-fa437d149e2a
Resolving doc-14-3c-docs.googleusercontent.com (doc-14-3c-docs.googleusercontent.com)... 142.250.99.132, 2607:

In [3]:
data=np.load('HandDB_Lower_Alphabet.npz', allow_pickle=True)
X=data['x']
y=data['y']
n2c=data['n2c'].item()